In [1]:
import pandas as pd
import numpy as np
import copy
import math

In [2]:
# Load the data into a pandas dataFrame
plays_2016 = pd.read_csv("datasets/plays2016_raw.csv")
plays_2016.head(4)

,Unnamed: 0,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,...,home_team,away_team,spread,offense_team,home_score,away_score,home_TO_rem,away_TO_rem,winner,box_url
0,0,Quarter,Time,Down,ToGo,Location,Detail,TAM,ATL,EPB,...,ATL,TAM,Atlanta Falcons -2.5,None,NaN,NaN,3,3,away,201609110atl
1,1,1st Quarter,NaN,0,0,NaN,NaN,NaN,NaN,NaN,...,ATL,TAM,Atlanta Falcons -2.5,None,ATL,TAM,3,3,away,201609110atl
2,2,1,15:00,NaN,NaN,TAM 35,"Roberto Aguayo kicks off 72 yards, returned by...",0,0,0.000,...,ATL,TAM,Atlanta Falcons -2.5,TAM,0,0,3,3,away,201609110atl
3,3,1,14:55,1,10,ATL 19,Devonta Freeman right tackle for 1 yard (tackl...,0,0,0.150,...,ATL,TAM,Atlanta Falcons -2.5,ATL,0,0,3,3,away,201609110atl


In [3]:
# input a couple of tables that might be useful
teams_long = [
    'Atlanta Falcons', 'New Orleans Saints', 'Carolina Panthers','Tampa Bay Buccaneers',
    'New York Giants', 'Dallas Cowboys', 'Philadelphia Eagles', 'Washington Redskins',
    'Minnesota Vikings', 'Green Bay Packers', 'Detroit Lions', 'Chicago Bears',
    'Seattle Seahawks', 'San Francisco 49ers', 'Arizona Cardinals',
    'St. Louis Rams', 'Los Angeles Rams',
    'Pittsburgh Steelers', 'Cleveland Browns', 'Cincinnati Bengals', 'Baltimore Ravens',
    'Buffalo Bills', 'New York Jets', 'Miami Dolphins', 'New England Patriots',
    'Tennessee Titans', 'Jacksonville Jaguars', 'Indianapolis Colts', 'Houston Texans',
    'Houston Oilers', 'Tennessee Oilers',
    'Denver Broncos', 'Oakland Raiders', 'San Diego Chargers', 'Kansas City Chiefs'
]

long_to_unique = {
    'Atlanta Falcons':'atl', 'New Orleans Saints':'nor', 
    'Carolina Panthers':'car', 'Tampa Bay Buccaneers':'tam',
    'New York Giants':'nyg', 'Dallas Cowboys':'dal',
    'Philadelphia Eagles':'phi', 'Washington Redskins':'was',
    'Minnesota Vikings':'min', 'Green Bay Packers':'gnb',
    'Detroit Lions':'det', 'Chicago Bears':'chi',
    'Seattle Seahawks':'sea', 'San Francisco 49ers':'sfo',
    'Arizona Cardinals':'crd', 'St. Louis Rams':'ram', 'Los Angeles Rams':'ram',
    'Pittsburgh Steelers':'pit', 'Cleveland Browns':'cle',
    'Baltimore Ravens':'rav', 'Cincinnati Bengals':'cin',
    'Buffalo Bills':'buf', 'New York Jets':'nyj',
    'Miami Dolphins':'mia', 'New England Patriots':'nwe',
    'Tennessee Titans':'oti', 'Tennessee Oilers':'oti', 'Jacksonville Jaguars':'jax',
    'Indianapolis Colts':'clt', 'Houston Texans':'htx',
    'Denver Broncos':'den', 'Oakland Raiders':'rai', 'Las Vegas Raiders':'rai',
    'San Diego Chargers':'sdg', 'Kansas City Chiefs':'kan'
}

long_to_short = {
    'Atlanta Falcons':'ATL', 'New Orleans Saints':'NOR', 
    'Carolina Panthers':'CAR', 'Tampa Bay Buccaneers':'TAM',
    'New York Giants':'NYG', 'Dallas Cowboys':'DAL',
    'Philadelphia Eagles':'PHI', 'Washington Redskins':'WAS',
    'Minnesota Vikings':'MIN', 'Green Bay Packers':'GNB',
    'Detroit Lions':'DET', 'Chicago Bears':'CHI',
    'Seattle Seahawks':'SEA', 'San Francisco 49ers':'SFO',
    'Arizona Cardinals':'ARI', 'St. Louis Rams':'STL', 'Los Angeles Rams':'LAR',
    'Pittsburgh Steelers':'PIT', 'Cleveland Browns':'CLE',
    'Baltimore Ravens':'BAL', 'Cincinnati Bengals':'CIN',
    'Buffalo Bills':'BUF', 'New York Jets':'NYJ',
    'Miami Dolphins':'MIA', 'New England Patriots':'NWE',
    'Tennessee Titans':'TEN', 'Tennessee Oilers':'TEN', 'Jacksonville Jaguars':'JAX',
    'Indianapolis Colts':'IND', 'Houston Texans':'HOU',
    'Denver Broncos':'DEN', 'Oakland Raiders':'OAK', 'Las Vegas Raiders':'LAR?',
    'San Diego Chargers':'SDG', 'Kansas City Chiefs':'KAN'
}

teams_short = [
    'atl','nor','car','tam','nyg','dal','phi','was',
    'min','gnb','det','chi','sea','sfo','crd','ram',
    'pit','cle','cin','rav','buf','nyj','mia','nwe',
    'oti','jax','clt','htx','den','rai','sdg','kan'
]

month = {
    'January':'01','February':'02','March':'03','April':'04','May':'05','June':'06',
    'July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'
}

place_to_team = {
    'TAM':'TAM', 'CAR':'CAR', 'NOR':'NOR', 'ATL':'ATL',
    'NYG':'NYG', 'DAL':'DAL', 'PHI':'PHI', 'WAS':'WAS',
    'MIN':'MIN', 'GNB':'GNB', 'DET':'DET', 'CHI':'CHI',
    'SEA':'SEA', 'SFO':'SFO', 'ARI':'CRD', 'LAR':'RAM', 'STL':'RAM',
    'PIT':'PIT', 'CLE':'CLE', 'CIN':'CIN', 'BAL':'RAV',
    'BUF':'BUF', 'NYJ':'NYJ', 'MIA':'MIA', 'NWE':'NWE',
    'TEN':'OTI', 'JAX':'JAX', 'IND':'CLT', 'HOU':'HTX',
    'OAK':'RAI', 'DEN':'DEN', 'SDG':'SDG', 'KAN':'KAN'
}

In [4]:
plays_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46326 entries, 0 to 46325
Data columns (total 25 columns):
Unnamed: 0          46326 non-null int64
quarter             46162 non-null object
qtr_time_remain     44851 non-null object
down                41328 non-null object
yds_to_go           41328 non-null object
location            43935 non-null object
detail              45048 non-null object
pbp_score_aw        44884 non-null object
pbp_score_hm        44884 non-null object
exp_pts_before      44884 non-null object
exp_pts_after       44884 non-null object
home_wp             43114 non-null object
players_involved    42172 non-null object
scoring_play        46326 non-null float64
change_of_poss      46326 non-null float64
home_team           46326 non-null object
away_team           46326 non-null object
spread              46326 non-null object
offense_team        46326 non-null object
home_score          45137 non-null object
away_score          45137 non-null object
home_TO_

### What columns do we want to use as input for the models?

- Home score
- Home team lead
- Home team has possession?
- Pre-game spread
- Time remaining in regulation
- Adjusted score differential
- Overtime?
- Field position
- Down
- Distance
- Home timeouts remaining
- Away timeouts remaining

#### Things that could add functionality, but don't have infrastructure to set up currently:

- Game stats so far

### Should we consider things from the perspective of the offense or the home team?

As far as I can reason, no real functional difference. So stick with what's easier.

In [ ]:
# Figure out how many timeouts are remaining for each team
# May want to update initial table grab with method for doing this

# Make column for home team's lead/deficit

# Make binary column for home team possession

# Make column for spread in terms of "home team favored by"

# Make column for time remaining in regulation

# Make binary column for overtime

# Make column for adjusted score

# Make column for field position (yardline from home team's perspective, -50 to +50)

In [5]:
# Drop rows for 1st/2nd Quarter, challenges, timeouts, etc.
def drop_nonplays(df):
    
    # Take out rows for the start of quarters/OT
    bool_list = [ not i for i in df['quarter'].str.contains("Quarter") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['quarter'].str.contains("End") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['quarter'].str.contains("Overtime") ]
    df = df.loc[ bool_list ]
    
    # Take out rows for Coach's challenges and timeouts
    bool_list = [ not i for i in df['detail'].str.contains("challenged") ]
    df = df.loc[ bool_list ]
    
    bool_list = [ not i for i in df['detail'].str.contains("Timeout") ]
    df = df.loc[ bool_list ]
    
    # Take out first play of further Overtime games
    
    return df

In [6]:
# Execute the previous function to drop non-plays
plays_2016 = drop_nonplays(plays_2016)

# Cast down and distance as integers, by first filling in empty values with zero
plays_2016['down'].fillna(0, inplace=True)
plays_2016['down'] = plays_2016.down.astype(int)
plays_2016['yds_to_go'].fillna(0, inplace=True)
plays_2016['yds_to_go'] = plays_2016.yds_to_go.astype(int)

# Filter to plays on downs 1-4
plays_2016 = plays_2016.query('down > 0')

In [7]:
# Make column for home team lead/deficit
plays_2016['home_score'].fillna(0, inplace=True)
plays_2016['away_score'].fillna(0, inplace=True)
plays_2016['home_score'] = plays_2016['home_score'].astype(int)
plays_2016['away_score'] = plays_2016['away_score'].astype(int)

plays_2016['home_lead'] = plays_2016['home_score'] - plays_2016['away_score']

In [8]:
plays_2016[['home_team','away_team','offense_team']].sample(5)

,home_team,away_team,offense_team
39046,TEN,OAK,OAK
27904,ARI,NYJ,ARI
15882,SEA,PHI,PHI
123,ATL,TAM,ATL
4829,NOR,LAR,LAR


In [9]:
# Make column for whether home team has possession
def home_poss(play):
    if play.offense_team == play.home_team:
        return 1
    elif play.offense_team == play.away_team:
        return 0
    else:
        print("match not found")
        return 0
plays_2016['home_possession'] = plays_2016.apply(
    lambda row: home_poss(row), axis=1 )

In [10]:
# Make column for spread in terms of "home team favored by X points"
favored_teams = [ 
    " ".join(str(i).split()[0:-1]) if str(i)!='Pick' else 'None' for i in plays_2016['spread'].values 
]
handicap = [ float(str(i).split()[-1]) if str(i)!='Pick' else 0 for i in plays_2016.spread.values ]

#def home_spread(play):
home_fav_by = []
for i, home_team in enumerate(plays_2016.home_team.values):
    try:
        if long_to_short[favored_teams[i]] in str(home_team):
            home_fav_by.append(-handicap[i])
        else:
            home_fav_by.append(handicap[i])
    except: # We most likely hit a 'Pick', so nobody's favored
        home_fav_by.append(0)
        
plays_2016['home_fav_by'] = home_fav_by

plays_2016[['spread','home_team','home_fav_by','home_possession']].sample(10)

,spread,home_team,home_fav_by,home_possession
10432,New York Giants -3.5,NYG,3.5,0
31155,Pittsburgh Steelers -3.0,PIT,3.0,1
33117,New York Jets -1.0,NYJ,1.0,0
20160,Green Bay Packers -6.0,GNB,6.0,0
3599,Carolina Panthers -2.5,NOR,-2.5,0
43116,Kansas City Chiefs -3.0,IND,-3.0,0
33747,New England Patriots -7.5,NWE,7.5,1
10287,Green Bay Packers -7.0,GNB,7.0,1
29539,Pittsburgh Steelers -10.0,PIT,10.0,1
6303,Carolina Panthers -6.0,CAR,6.0,0


In [11]:
# Make column for time remaining in regulation
def get_secs_remaining(play):
    try:
        if "OT" in str(play.quarter):
            qtr = 4
        else:
            qtr = int(play.quarter)
    except:
        qtr = 4
        
    try:
        mins_rem, secs_rem = str(play.qtr_time_remain).split(":")
    except:
        mins_rem = 15
        secs_rem = 0
        
    secs_played = 900*(qtr-1) + 900 - ( 60*int(mins_rem) + int(secs_rem) )
    return 3600-secs_played

plays_2016['secs_remaining'] = plays_2016.apply( 
        lambda row: get_secs_remaining(row), axis=1
    )

In [12]:
# Make binary column for whether game is in OT
plays_2016['OT'] = [ 1 if 'OT' in str(i) else 0 for i in plays_2016.quarter.values ]

In [13]:
# Make a column for adjusted score
def adjusted_lead(play):
    if play.OT == 1:
        return 0
    elif play.secs_remaining <= 3600:
        return play.home_lead / math.sqrt( 3600-play.secs_remaining + 1 )
plays_2016['adj_lead'] = plays_2016.apply(
    lambda row: adjusted_lead(row), axis=1 )

In [21]:
def home_loc(df):

    # Make a column for home team field position
    fieldside = [ str(i).split()[0] for i in df.location.values ]
    fieldpos = [ int(str(i).split()[-1]) for i in df.location.values ]

    loc = []
    for i, hometeam in enumerate(df.home_team.values):
        if place_to_team[hometeam] in fieldside[i]:
            # Ball is on home team's side
            loc.append(-(50-fieldpos[i]))
        else:
            loc.append(50-fieldpos[i])
        
    return loc
    

# Call the function
plays_2016['home_fieldpos'] = home_loc(plays_2016)

plays_2016[['home_team','location','home_fieldpos']].sample(20)

,home_team,location,home_fieldpos
10185,MIN,MIN 46,-4
1584,TAM,ATL 48,2
20879,GNB,CLT 43,7
6989,OAK,RAI 1,-49
16180,CIN,PHI 25,25
19082,CHI,CHI 37,-13
29090,PIT,CIN 42,8
19284,MIN,CRD 44,6
13229,GNB,DAL 38,12
18803,MIN,HTX 1,49


In [15]:
plays_2016[['home_team','winner']].sample(5)

,home_team,winner
18799,MIN,home
36414,BUF,home
24363,LAR,home
25965,SFO,home
20402,JAX,away


In [16]:
# Make a binary column for "home team wins"
plays_2016['home_win'] = [1 if 'home' in str(i) else 0 for i in plays_2016['winner'].values]
plays_2016[['home_team','winner','home_win']].sample(5)

,home_team,winner,home_win
35983,MIA,home,1
31272,CLE,away,0
6347,CAR,home,1
29866,PIT,away,0
17317,DET,home,1


In [17]:
plays_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38172 entries, 3 to 46324
Data columns (total 33 columns):
Unnamed: 0          38172 non-null int64
quarter             38172 non-null object
qtr_time_remain     38172 non-null object
down                38172 non-null int32
yds_to_go           38172 non-null int32
location            38172 non-null object
detail              38172 non-null object
pbp_score_aw        38172 non-null object
pbp_score_hm        38172 non-null object
exp_pts_before      38172 non-null object
exp_pts_after       38172 non-null object
home_wp             38172 non-null object
players_involved    38110 non-null object
scoring_play        38172 non-null float64
change_of_poss      38172 non-null float64
home_team           38172 non-null object
away_team           38172 non-null object
spread              38172 non-null object
offense_team        38172 non-null object
home_score          38172 non-null int32
away_score          38172 non-null int32
home_TO_rem 

In [18]:
x_cols = ['home_fav_by', 'secs_remaining', 'OT', 'home_score', 'home_lead', 'adj_lead',
          'home_possession', 'home_fieldpos', 'down', 'yds_to_go', 'home_TO_rem', 'away_TO_rem']
y_col = 'home_win'

In [19]:
plays2016_modelinput = plays_2016[x_cols+[y_col]]
plays2016_modelinput.sample(10)

,home_fav_by,secs_remaining,OT,home_score,home_lead,adj_lead,home_possession,home_fieldpos,down,yds_to_go,home_TO_rem,away_TO_rem,home_win
26373,7.5,2561,0,7,-3,-0.093026,1,21,1,10,3,2,1
20887,7.5,3346,0,3,-4,-0.250490,0,25,1,10,3,3,0
2470,13.5,2076,0,21,14,0.358503,1,41,1,9,2,2,1
12374,9.0,1068,0,0,-10,-0.198693,0,39,4,4,3,3,0
33798,2.5,3197,0,0,-7,-0.348263,1,-29,2,14,3,3,0
21624,-4.5,2700,0,0,0,0.000000,1,-16,4,4,2,3,0
24127,10.5,3056,0,0,0,0.000000,1,34,3,9,3,3,1
42740,1.5,549,0,20,1,0.018101,0,-40,1,10,3,3,1
43671,-1.0,896,0,13,7,0.134590,1,-10,1,10,3,3,1
23489,4.0,1511,0,16,3,0.065622,1,-10,2,10,3,3,1


In [20]:
plays2016_modelinput.to_csv("datasets/2016plays_modelinput.csv")

In [36]:
def munge_df(df):
    # Nice function for doing all of the required munging in one step
    
    # Drop rows that aren't plays
    df = drop_nonplays(df)
    
    # Cast down and distance as integers, by first filling in empty values with zero
    df['down'].fillna(0, inplace=True)
    df['down'] = df.down.astype(int)
    df['yds_to_go'].fillna(0, inplace=True)
    df['yds_to_go'] = df.yds_to_go.astype(int)

    # Filter to plays on downs 1-4
    df = df.loc[ df['down'] > 0 , : ]
    
    # Make column for home team lead/deficit
    df['home_score'].fillna(0, inplace=True)
    df['away_score'].fillna(0, inplace=True)
    df['home_score'] = df['home_score'].astype(int)
    df['away_score'] = df['away_score'].astype(int)
    df['home_lead'] = df['home_score'] - df['away_score']
    
    # Make binary column for whether home team has possession at beginning of play
    df['home_possession'] = df.apply( lambda row: home_poss(row), axis=1 )
    
    
    # Make column for spread in terms of "home team favored by X points"
    favored_teams = [ " ".join(str(i).split()[0:-1]) if str(i)!='Pick' 
                      else 'None' for i in df['spread'].values 
                    ]
    handicap = [ float(str(i).split()[-1]) if str(i)!='Pick' 
                 else 0 for i in df.spread.values 
               ]
  
    # Get the spread in form of "home team is favored by X points"
    home_fav_by = []
    for i, home_team in enumerate(df.home_team.values):
        try:
            if long_to_short[favored_teams[i]] in str(home_team):
                home_fav_by.append(-handicap[i])
            else:
                home_fav_by.append(handicap[i])
        except: # We most likely hit a 'Pick', so nobody's favored
            home_fav_by.append(0)
        
    df['home_fav_by'] = home_fav_by
    
    
    # Get number of seconds remaining in regulation/OT for each play
    df['secs_remaining'] = df.apply( 
        lambda row: get_secs_remaining(row), axis=1 )
    
    # Make binary column for whether game is in OT
    df['OT'] = [ 1 if 'OT' in str(i) else 0 for i in df.quarter.values ]
    
    
    # Make column for home team's adjusted lead
    df['adj_lead'] = df.apply( lambda row: adjusted_lead(row), axis=1 )
    
    
    # Make column for field position from home team's perspective
    df['home_fieldpos'] = home_loc(df)
    
    
    # Make a binary column for "home team wins"
    df['home_win'] = [1 if 'home' in str(i) else 0 for i in df['winner'].values]
    
    
    # And finally return just the columns we need for model input
    return df[x_cols+[y_col]]

In [40]:
# Loop to make model-ready dataframes for years 2000-2015
for i in range(2013,2016):
    print("")
    print(i)
    year_df = pd.read_csv("datasets/plays{0}_raw.csv".format(i))
    year_df = munge_df(year_df)
    print(year_df.sample(5))
    year_df.to_csv("datasets/plays{0}_modelready.csv".format(i))


2013
       home_fav_by  secs_remaining  OT  home_score  home_lead  adj_lead  \
14404          1.5            2350   0           7          0  0.000000   
10127         -4.0            2656   0           3         -4 -0.130120   
9364           2.5            3453   0           7          7  0.575396   
573            3.0            2700   0           0          0  0.000000   
9192           7.0            3264   0           0          0  0.000000   

       home_possession  home_fieldpos  down  yds_to_go  home_TO_rem  \
14404                1             43     2          7            3   
10127                1            -28     1         10            2   
9364                 0              4     2          6            3   
573                  1            -40     2          6            3   
9192                 0             33     3         15            3   

       away_TO_rem  home_win  
14404            3         1  
10127            3         0  
9364             3     

In [41]:
# Debug method for 2003 and 2012 sets
plays_2012 = pd.read_csv("datasets/plays2012_raw.csv".format(i))

In [43]:
plays_2012 = munge_df(plays_2012)

ValueError: invalid literal for int() with base 10: ' DEN'

In [67]:
bool_list = [ True if 'DEN' in str(i) else False for i in plays_2012.home_score.values ]
plays_2012.loc[ bool_list, ['detail','quarter','home_team','away_team','pbp_score_aw','home_score','box_url'] ]

,detail,quarter,home_team,away_team,pbp_score_aw,home_score,box_url
4187,NaN,1st Quarter,DEN,NOR,NaN,DEN,201210280den
4231,NaN,2nd Quarter,DEN,NOR,NaN,DEN,201210280den
4277,NaN,3rd Quarter,DEN,NOR,NaN,DEN,201210280den
4315,NaN,4th Quarter,DEN,NOR,NaN,DEN,201210280den
8923,NaN,1st Quarter,DEN,TAM,NaN,DEN,201212020den
8968,NaN,2nd Quarter,DEN,TAM,NaN,DEN,201212020den
9007,NaN,3rd Quarter,DEN,TAM,NaN,DEN,201212020den
9061,NaN,4th Quarter,DEN,TAM,NaN,DEN,201212020den
28040,NaN,1st Quarter,DEN,PIT,NaN,DEN,201209090den
28075,NaN,2nd Quarter,DEN,PIT,NaN,DEN,201209090den


In [61]:
plays_2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44682 entries, 0 to 44681
Data columns (total 25 columns):
Unnamed: 0          44682 non-null int64
quarter             44285 non-null object
qtr_time_remain     42951 non-null object
down                38948 non-null object
yds_to_go           38948 non-null object
location            40253 non-null object
detail              43446 non-null object
pbp_score_aw        43053 non-null object
pbp_score_hm        43053 non-null object
exp_pts_before      43053 non-null object
exp_pts_after       43053 non-null object
home_wp             41389 non-null object
players_involved    40604 non-null object
scoring_play        44682 non-null float64
change_of_poss      44682 non-null float64
home_team           44682 non-null object
away_team           44682 non-null object
spread              44682 non-null object
offense_team        44682 non-null object
home_score          43291 non-null object
away_score          43291 non-null object
home_TO_